# Семинар №2
## Выполнил: Поверинов И.В.
## Группа: ИДБ-22-10
## Задание: 2

1. Применить методы линейной регрессии к заданию из семинара №1 (к столбцу price)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

data = pd.read_csv('cars.csv')
print(data.isnull().sum())
print(f"\nКоличество дубликатов: {data.duplicated().sum()}")

mileage         0
make            0
model         143
fuel          247
gear          182
offerType       0
price        1533
hp             29
year            0
dtype: int64

Количество дубликатов: 2003


In [2]:
pd.set_option('future.no_silent_downcasting', True)
data = data.rename(columns={'offerType': 'offer_type'})
cleanData = data.drop_duplicates()
cleanData = cleanData.reset_index(drop = True)
def missing_values(column):
    dtype = column.dtype
    if len(column) == 0:
        return column
    if dtype == 'object':
        return column.fillna(column.mode().iloc[0]).infer_objects(copy=False)
    elif pd.api.types.is_numeric_dtype(dtype):
        return column.fillna(column.median())
    else:
        return column.interpolate(method='linear')


unique_make = cleanData['make'].unique()
maken = dict(zip(unique_make, range(len(unique_make))))
cleanData.loc[:, 'make'] = cleanData['make'].map(maken)
cleanData.loc[:, 'make'] = cleanData['make'].astype(int)

unique_model = cleanData['model'].unique()
modell = dict(zip(unique_model, range(len(unique_model))))
cleanData.loc[:, 'model'] = cleanData['model'].map(modell)
cleanData.loc[:, 'model'] = cleanData['model'].astype(int)

unique_fuel = cleanData['fuel'].unique()
fuell = dict(zip(unique_fuel, range(len(unique_fuel))))
cleanData.loc[:, 'fuel'] = cleanData['fuel'].map(fuell)
cleanData.loc[:, 'fuel'] = cleanData['fuel'].astype(int)

unique_gear = cleanData['gear'].unique()
gearl = dict(zip(unique_gear, range(len(unique_gear))))
cleanData.loc[:, 'gear'] = cleanData['gear'].map(gearl)
cleanData.loc[:, 'gear'] = cleanData['gear'].astype(int)

unique_offer = cleanData['offer_type'].unique()
offerl = dict(zip(unique_offer, range(len(unique_offer))))
cleanData.loc[:, 'offer_type'] = cleanData['offer_type'].map(offerl)
cleanData.loc[:, 'offer_type'] = cleanData['offer_type'].astype(int)

def remove_outlier(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    return df[~((df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR)))]
cleanData = remove_outlier(cleanData, 'price')

cleanData.loc[:, 'model'] = missing_values(cleanData.loc[:, 'model'])
cleanData.loc[:, 'fuel'] = missing_values(cleanData.loc[:, 'fuel'])
cleanData.loc[:, 'gear'] = missing_values(cleanData.loc[:, 'gear'])

original = cleanData.copy()

cleanData = cleanData.dropna(subset=['hp', 'price'])
cleanData = cleanData.drop_duplicates()

print(cleanData.isnull().sum())
print(f"\nКоличество дубликатов: {cleanData.duplicated().sum()}")
print(cleanData.head(10))

mileage       0
make          0
model         0
fuel          0
gear          0
offer_type    0
price         0
hp            0
year          0
dtype: int64

Количество дубликатов: 0
    mileage make model fuel gear offer_type   price     hp  year
0    235000    0     0    0    0          0  6800.0  116.0  2011
1     92800    1     1    1    0          0  6877.0  122.0  2011
2    149300    2     2    1    0          0  6900.0  160.0  2011
3     96200    3     3    1    0          0  6950.0  110.0  2011
4    156000    4     4    1    0          0  6950.0  156.0  2011
5    147000    5     5    2    1          0  6950.0   99.0  2011
6     91894    3     6    0    0          0  6970.0  131.0  2011
8    115000    7     8    1    1          0  6980.0  150.0  2011
9       104    8     9    0    0          0  6990.0   86.0  2011
10    59580    6    10    0    1          0  6990.0  101.0  2011


In [3]:

cleanData['price'] = cleanData['price'].fillna(cleanData['price'].median())
features = cleanData.drop(['price'], axis=1)
target = cleanData['price']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
pred_lr_test = model.predict(X_test)
pred_lr_train = model.predict(X_train)

rmse_lr_test = mean_squared_error(y_test, pred_lr_test) ** 0.5
rmse_lr_train = mean_squared_error(y_train, pred_lr_train) ** 0.5

print(f"LNR RMSE Test: {rmse_lr_test}")
print(f"LNR RMSE Train: {rmse_lr_train}")
print(f"LNR Difference: {abs(rmse_lr_train - rmse_lr_test)}\n")

print(f"Количество пропущенных значений в исходном DataFrame: {original['price'].isna().sum()}\n" )

missing_prices = original[original['price'].isna()].index
features_original = original.loc[missing_prices].drop(['price'], axis=1)
features_original = features_original.fillna(features_original.mean())
predicted_prices = model.predict(features_original)
original.loc[missing_prices, 'price'] = predicted_prices

print(f"Количество предсказанных цен: {len(predicted_prices)}\n")
print(f"Количество предсказанных цен для оригинальных пропущенных значений: {len(missing_prices)}\n")
print("Первые несколько предсказанных цен:")
for idx in missing_prices[:5]:
    print(f"{idx}: {original.at[idx, 'price']:.2f}")

LNR RMSE Test: 3282.1461467924955
LNR RMSE Train: 3269.9306454652856
LNR Difference: 12.21550132720995

Количество пропущенных значений в исходном DataFrame: 1520

Количество предсказанных цен: 1520

Количество предсказанных цен для оригинальных пропущенных значений: 1520

Первые несколько предсказанных цен:
7: 4801.51
35: 4378.97
119: 28375.65
137: 26307.74
181: 31691.95
